# Azure AI Search με Ενσωμάτωση NVIDIA NIM και LlamaIndex

Σε αυτό το σημειωματάριο, θα δείξουμε πώς να αξιοποιήσουμε τα μοντέλα AI της NVIDIA και το LlamaIndex για τη δημιουργία μιας ισχυρής διαδικασίας Retrieval-Augmented Generation (RAG). Θα χρησιμοποιήσουμε τα LLMs και τα embeddings της NVIDIA, θα τα ενσωματώσουμε με το Azure AI Search ως αποθηκευτικό χώρο διανυσμάτων και θα εκτελέσουμε RAG για τη βελτίωση της ποιότητας και της αποτελεσματικότητας της αναζήτησης.

## Οφέλη
- **Κλιμακωσιμότητα**: Αξιοποιήστε τα μεγάλα γλωσσικά μοντέλα της NVIDIA και το Azure AI Search για κλιμακούμενη και αποτελεσματική ανάκτηση.
- **Αποδοτικότητα Κόστους**: Βελτιστοποιήστε την αναζήτηση και την ανάκτηση με αποδοτική αποθήκευση διανυσμάτων και υβριδικές τεχνικές αναζήτησης.
- **Υψηλή Απόδοση**: Συνδυάστε ισχυρά LLMs με αναζήτηση βασισμένη σε διανύσματα για ταχύτερες και πιο ακριβείς απαντήσεις.
- **Ποιότητα**: Διατηρήστε υψηλή ποιότητα αναζήτησης, βασίζοντας τις απαντήσεις των LLM σε σχετικά έγγραφα που έχουν ανακτηθεί.

## Προαπαιτούμενα
- 🐍 Python 3.9 ή νεότερη έκδοση
- 🔗 [Υπηρεσία Azure AI Search](https://learn.microsoft.com/azure/search/)
- 🔗 Κλειδί API της NVIDIA για πρόσβαση στα LLMs και Embeddings της NVIDIA μέσω των μικροϋπηρεσιών NVIDIA NIM

## Καλυπτόμενες Λειτουργίες
- ✅ Ενσωμάτωση LLM της NVIDIA (θα χρησιμοποιήσουμε το [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ Embeddings της NVIDIA (θα χρησιμοποιήσουμε το [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ Προηγμένες Λειτουργίες Ανάκτησης του Azure AI Search
- ✅ Ευρετηρίαση Εγγράφων με LlamaIndex
- ✅ RAG χρησιμοποιώντας το Azure AI Search και το LlamaIndex με LLMs της NVIDIA

Ας ξεκινήσουμε!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## Εγκατάσταση και Απαιτήσεις
Δημιουργήστε ένα περιβάλλον Python χρησιμοποιώντας έκδοση Python >3.10.

## Ξεκινώντας!


Για να ξεκινήσετε, χρειάζεστε ένα `NVIDIA_API_KEY` για να χρησιμοποιήσετε τα μοντέλα NVIDIA AI Foundation:
1) Δημιουργήστε έναν δωρεάν λογαριασμό με [NVIDIA](https://build.nvidia.com/explore/discover).
2) Κάντε κλικ στο μοντέλο της επιλογής σας.
3) Στην ενότητα Εισόδου, επιλέξτε την καρτέλα Python και κάντε κλικ στο **Get API Key** και στη συνέχεια στο **Generate Key**.
4) Αντιγράψτε και αποθηκεύστε το παραγόμενο κλειδί ως NVIDIA_API_KEY. Από εκεί, θα έχετε πρόσβαση στα endpoints.


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## Παράδειγμα RAG χρησιμοποιώντας LLM και Ενσωμάτωση
### 1) Αρχικοποίηση του LLM
Το `llama-index-llms-nvidia`, γνωστό και ως συνδετήρας LLM της NVIDIA, σας επιτρέπει να συνδεθείτε και να δημιουργήσετε από συμβατά μοντέλα που είναι διαθέσιμα στον κατάλογο API της NVIDIA. Δείτε εδώ για μια λίστα με μοντέλα ολοκλήρωσης συνομιλίας: https://build.nvidia.com/search?term=Text-to-Text

Εδώ θα χρησιμοποιήσουμε το **mixtral-8x7b-instruct-v0.1**


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) Αρχικοποίηση του Embedding  
Το `llama-index-embeddings-nvidia`, γνωστό και ως ο συνδετήρας Embeddings της NVIDIA, σας επιτρέπει να συνδεθείτε και να δημιουργήσετε από συμβατά μοντέλα που είναι διαθέσιμα στον κατάλογο API της NVIDIA. Επιλέξαμε το `nvidia/nv-embedqa-e5-v5` ως το μοντέλο embedding. Δείτε εδώ μια λίστα με μοντέλα ενσωμάτωσης κειμένου: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### 3) Δημιουργήστε ένα Azure AI Search Vector Store


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) Δημιουργία Μηχανής Ερωτήσεων για να κάνετε ερωτήσεις σχετικά με τα δεδομένα σας

Ακολουθεί ένα ερώτημα που χρησιμοποιεί καθαρή αναζήτηση με διανύσματα στο Azure AI Search και βασίζει την απάντηση στο LLM μας (Phi-3.5-MOE)


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Εδώ είναι ένα ερώτημα χρησιμοποιώντας υβριδική αναζήτηση στο Azure AI Search.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### Ανάλυση Αναζήτησης Vector
Η απάντηση του LLM αποτυπώνει με ακρίβεια τις βασικές οικονομικές συνέπειες που αναφέρονται στο αρχικό κείμενο σχετικά με τη χρηματιστηριακή αγορά της Ρωσίας. Συγκεκριμένα, αναφέρει ότι η χρηματιστηριακή αγορά της Ρωσίας υπέστη σημαντική πτώση, χάνοντας το 40% της αξίας της, και ότι οι συναλλαγές ανεστάλησαν λόγω της τρέχουσας κατάστασης. Αυτή η απάντηση ευθυγραμμίζεται καλά με τις πληροφορίες που παρέχονται στο αρχικό κείμενο, υποδεικνύοντας ότι το LLM εντόπισε και συνοψίζει σωστά τις σχετικές λεπτομέρειες σχετικά με τον αντίκτυπο στη χρηματιστηριακή αγορά ως αποτέλεσμα των ενεργειών της Ρωσίας και των επιβληθέντων κυρώσεων.

#### Σχόλια για τους Κόμβους Πηγής
Οι κόμβοι πηγής παρέχουν μια λεπτομερή περιγραφή των οικονομικών συνεπειών που αντιμετώπισε η Ρωσία λόγω των διεθνών κυρώσεων. Το κείμενο τονίζει ότι η χρηματιστηριακή αγορά της Ρωσίας έχασε το 40% της αξίας της και ότι οι συναλλαγές ανεστάλησαν. Επιπλέον, αναφέρει άλλες οικονομικές επιπτώσεις, όπως την υποτίμηση του Ρουβλίου και τη γενικότερη απομόνωση της οικονομίας της Ρωσίας. Η απάντηση του LLM απέδωσε αποτελεσματικά τα βασικά σημεία από αυτούς τους κόμβους, εστιάζοντας στον αντίκτυπο στη χρηματιστηριακή αγορά, όπως ζητήθηκε από την ερώτηση.


Τώρα, ας δούμε ένα ερώτημα όπου η Υβριδική Αναζήτηση δεν δίνει μια καλά τεκμηριωμένη απάντηση:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### Υβριδική Αναζήτηση: Ανάλυση Απόκρισης LLM
Η απόκριση του LLM στο παράδειγμα Υβριδικής Αναζήτησης υποδεικνύει ότι το παρεχόμενο πλαίσιο δεν καθορίζει την ακριβή ημερομηνία της εισβολής της Ρωσίας στην Ουκρανία. Αυτή η απόκριση δείχνει ότι το LLM αξιοποιεί τις διαθέσιμες πληροφορίες από τα έγγραφα πηγής, ενώ αναγνωρίζει την απουσία συγκεκριμένων λεπτομερειών στο κείμενο.

Η απόκριση είναι ακριβής στο να εντοπίζει ότι το πλαίσιο αναφέρει γεγονότα σχετικά με την επιθετικότητα της Ρωσίας, αλλά δεν προσδιορίζει την συγκεκριμένη ημερομηνία της εισβολής. Αυτό αναδεικνύει την ικανότητα του LLM να κατανοεί τις παρεχόμενες πληροφορίες, ενώ ταυτόχρονα αναγνωρίζει κενά στο περιεχόμενο. Το LLM προτρέπει αποτελεσματικά τον χρήστη να αναζητήσει εξωτερικές πηγές ή ιστορικά αρχεία για την ακριβή ημερομηνία, επιδεικνύοντας ένα επίπεδο προσοχής όταν οι πληροφορίες είναι ελλιπείς.

### Ανάλυση Κόμβων Πηγής
Οι κόμβοι πηγής στο παράδειγμα Υβριδικής Αναζήτησης περιέχουν αποσπάσματα από μια ομιλία που συζητά την αντίδραση των Η.Π.Α. στις ενέργειες της Ρωσίας στην Ουκρανία. Αυτοί οι κόμβοι δίνουν έμφαση στον ευρύτερο γεωπολιτικό αντίκτυπο και στα βήματα που έλαβαν οι Η.Π.Α. και οι σύμμαχοί τους ως απάντηση στην εισβολή, αλλά δεν αναφέρουν την συγκεκριμένη ημερομηνία της εισβολής. Αυτό συμφωνεί με την απόκριση του LLM, η οποία σωστά εντοπίζει ότι το πλαίσιο στερείται της ακριβούς πληροφορίας για την ημερομηνία.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### Υβριδικό με Επανακατάταξη: Ανάλυση Απόκρισης LLM
Στο παράδειγμα Υβριδικού με Επανακατάταξη, η απόκριση του LLM παρέχει επιπλέον πλαίσιο σημειώνοντας ότι το γεγονός συνέβη έξι ημέρες πριν από την ομιλία. Αυτό υποδεικνύει ότι το LLM είναι σε θέση να συμπεράνει την ημερομηνία της εισβολής με βάση το χρονικό σημείο της ομιλίας, παρόλο που εξακολουθεί να απαιτεί τη γνώση της ακριβούς ημερομηνίας της ομιλίας για ακρίβεια.

Αυτή η απόκριση δείχνει μια βελτιωμένη ικανότητα χρήσης στοιχείων από το πλαίσιο για την παροχή μιας πιο ενημερωτικής απάντησης. Τονίζει το πλεονέκτημα της επανακατάταξης, όπου το LLM μπορεί να έχει πρόσβαση και να δίνει προτεραιότητα σε πιο σχετικές πληροφορίες για να προσφέρει μια πιο κοντινή προσέγγιση της επιθυμητής λεπτομέρειας (δηλαδή, της ημερομηνίας της εισβολής).

### Ανάλυση Κόμβων Πηγής
Οι κόμβοι πηγής σε αυτό το παράδειγμα περιλαμβάνουν αναφορές στο χρονικό σημείο της εισβολής της Ρωσίας, συγκεκριμένα αναφέροντας ότι συνέβη έξι ημέρες πριν από την ομιλία. Παρόλο που η ακριβής ημερομηνία δεν δηλώνεται ρητά, οι κόμβοι παρέχουν χρονικό πλαίσιο που επιτρέπει στο LLM να δώσει μια πιο λεπτομερή απάντηση. Η συμπερίληψη αυτής της λεπτομέρειας αναδεικνύει πώς η επανακατάταξη μπορεί να βελτιώσει την ικανότητα του LLM να εξάγει και να συμπεραίνει πληροφορίες από το παρεχόμενο πλαίσιο, οδηγώντας σε μια πιο ακριβή και ενημερωτική απάντηση.


**Σημείωση:**
Σε αυτό το σημειωματάριο, χρησιμοποιήσαμε μικροϋπηρεσίες NVIDIA NIM από τον κατάλογο API της NVIDIA.  
Οι παραπάνω APIs, `NVIDIA (llms)`, `NVIDIAEmbedding`, και [Azure AI Search Semantic Hybrid Retrieval (built-in reranking)](https://learn.microsoft.com/azure/search/semantic-search-overview). Σημειώστε ότι οι παραπάνω APIs μπορούν επίσης να υποστηρίξουν μικροϋπηρεσίες που φιλοξενούνται τοπικά.

**Παράδειγμα:**
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")```



---

**Αποποίηση ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης AI [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε προσπάθειες για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτόματες μεταφράσεις ενδέχεται να περιέχουν λάθη ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
